
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 16.50 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

/tmp/ipykernel_11320/160627683.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-29 01:32:36.797626: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 01:32:36.865182: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 01:32:36.865227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 01:32:36.866810: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 01:32:36.875863: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 01:32:36.888700: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[30, 15], [45, 30], [60, 30]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[30, 15], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  1.6115531699997518
Stopped at epoch:  26
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.5285670580005899
Stopped at epoch:  12
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.3042734840000776
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.32325043700075184
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.42741355399994063
Stopped at epoch:  9
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.2674108270002762
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 7 +++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_9_3,0.801239,0.821812,0.708200,0.777230,0.219970,0.214162,0.441506,0.321147
model_1_9_1,0.801333,0.819848,0.718786,0.781591,0.219866,0.216523,0.425489,0.314861
model_1_9_2,0.801340,0.823162,0.709885,0.778658,0.219858,0.212540,0.438957,0.319090
model_1_9_0,0.801754,0.833517,0.728327,0.792337,0.219400,0.200094,0.411053,0.299369
model_1_9_4,0.801866,0.814259,0.706977,0.773292,0.219276,0.223241,0.443357,0.326825


DataFrame salvo em ../results/metrics_1_9
Testando combinacao2: Hidden Size=[30, 15], regularizer=0.02, learning_rate=0.1
+++++++++++ [2_0] | 1 ++++++++++++++++++
Elapsed time:  2.0115853580000476
Stopped at epoch:  29
1/1 [==============================] - 0s 14ms/step
+++++++++++ [2_0] | 2 ++++++++++++++++++
Elapsed time:  0.3974547320003694
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_0] | 3 ++++++++++++++++++
Elapsed time:  0.8672243539995179
Stopped at epoch:  13
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_0] | 4 ++++++++++++++++++
Elapsed time:  0.26597804500033817
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [2_0] | 5 ++++++++++++++++++
Elapsed time:  0.5746648399999685
Stopped at epoch:  9
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_0] | 6 ++++++++++++++++++
Elapsed time:  0.7627739129993643
Stopped at epoch:  14
1/1 [==============================] -

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_2_0,0.800358,0.412121,-2.380189,0.345238,0.220945,0.799986,0.561935,0.687963
model_2_2_1,0.804331,0.432294,-2.407805,0.357014,0.216548,0.772534,0.566526,0.675589
model_2_2_2,0.806763,0.446465,-2.430822,0.365017,0.213857,0.753250,0.570353,0.667181
model_2_2_3,0.806974,0.449844,-2.461458,0.365052,0.213622,0.748652,0.575446,0.667143
model_2_2_5,0.808488,0.462900,-2.517862,0.369804,0.211947,0.730886,0.584823,0.662150
model_2_2_4,0.808602,0.460588,-2.491074,0.370214,0.211821,0.734031,0.580369,0.661720
model_2_2_6,0.808800,0.462727,-2.506584,0.370526,0.211602,0.731121,0.582948,0.661393
model_2_2_7,0.812265,0.476069,-2.468424,0.382515,0.207767,0.712966,0.576604,0.648795
model_2_2_9,0.812402,0.478512,-2.499464,0.381879,0.207616,0.709641,0.581764,0.649464
model_2_2_8,0.812898,0.479489,-2.482423,0.383818,0.207067,0.708311,0.578931,0.647426


DataFrame salvo em ../results/metrics_2_2
+++++++++++ [2_3] | 1 ++++++++++++++++++
Elapsed time:  0.3420501720001994
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_3] | 2 ++++++++++++++++++
Elapsed time:  0.45150124200063146
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [2_3] | 3 ++++++++++++++++++
Elapsed time:  0.43998496400035947
Stopped at epoch:  7
1/1 [==============================] - 0s 22ms/step
+++++++++++ [2_3] | 4 ++++++++++++++++++
Elapsed time:  0.8441037689999575
Stopped at epoch:  14
1/1 [==============================] - 0s 21ms/step
+++++++++++ [2_3] | 5 ++++++++++++++++++
Elapsed time:  0.4700767590002215
Stopped at epoch:  9
1/1 [==============================] - 0s 19ms/step
+++++++++++ [2_3] | 6 ++++++++++++++++++
Elapsed time:  0.35954602399942814
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [2_3] | 7 ++++++++++++++++++
Elapsed time:  0.40641588

...

DataFrame salvo em ../results/metrics_11_8
+++++++++++ [11_9] | 1 ++++++++++++++++++
Elapsed time:  1.1048196709998592
Stopped at epoch:  17
1/1 [==============================] - 0s 16ms/step
+++++++++++ [11_9] | 2 ++++++++++++++++++
Elapsed time:  0.6080478479998419
Stopped at epoch:  10
1/1 [==============================] - 0s 29ms/step
+++++++++++ [11_9] | 3 ++++++++++++++++++
Elapsed time:  0.5021616789999825
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [11_9] | 4 ++++++++++++++++++
Elapsed time:  0.4966755059995194
Stopped at epoch:  6
1/1 [==============================] - 0s 31ms/step
+++++++++++ [11_9] | 5 ++++++++++++++++++
Elapsed time:  0.4590032690002772
Stopped at epoch:  6
1/1 [==============================] - 0s 26ms/step
+++++++++++ [11_9] | 6 ++++++++++++++++++
Elapsed time:  0.471734581000419
Stopped at epoch:  6
1/1 [==============================] - 0s 26ms/step
+++++++++++ [11_9] | 7 ++++++++++++++++++
Elapsed time:  0.476

2024-02-29 02:05:52.289225: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


1/1 [==============================] - 0s 25ms/step
+++++++++++ [11_9] | 19 ++++++++++++++++++
Elapsed time:  0.5716369580004539
Stopped at epoch:  6
1/1 [==============================] - 0s 30ms/step
+++++++++++ [11_9] | 20 ++++++++++++++++++
Elapsed time:  0.44806968699958816
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [11_9] | 21 ++++++++++++++++++
Elapsed time:  0.5132261329999892
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [11_9] | 22 ++++++++++++++++++
Elapsed time:  0.4745037170005162
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [11_9] | 23 ++++++++++++++++++
Elapsed time:  0.5201684910007316
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [11_9] | 24 ++++++++++++++++++
Elapsed time:  0.4760550780001722
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [11_9] | 25 ++++++++++++++++++
Elap

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_11_9_24,0.923272,0.394703,0.798181,0.758634,0.084916,0.157135,0.400650,0.271731
model_11_9_23,0.924039,0.436698,0.797236,0.762978,0.084066,0.146233,0.402526,0.266841
model_11_9_22,0.928748,0.518047,0.805515,0.779778,0.078855,0.125115,0.386091,0.247928
model_11_9_21,0.929788,0.557692,0.806868,0.785740,0.077704,0.114823,0.383406,0.241215
model_11_9_17,0.929979,0.604435,0.810449,0.794418,0.077493,0.102689,0.376295,0.231445
model_11_9_16,0.931098,0.593500,0.822002,0.802671,0.076255,0.105527,0.353362,0.222155
model_11_9_20,0.931333,0.567349,0.811679,0.790911,0.075994,0.112316,0.373856,0.235394
model_11_9_18,0.931698,0.596868,0.813115,0.795705,0.075590,0.104653,0.371005,0.229996
model_11_9_19,0.931813,0.580975,0.811964,0.792812,0.075463,0.108779,0.373288,0.233253
model_11_9_15,0.936546,0.672571,0.837668,0.825324,0.070225,0.085001,0.322261,0.196652


DataFrame salvo em ../results/metrics_11_9
Testando combinacao12: Hidden Size=[60, 30], regularizer=0.1, learning_rate=0.1
+++++++++++ [12_0] | 1 ++++++++++++++++++
Elapsed time:  0.7096352619992103
Stopped at epoch:  0
1/1 [==============================] - 0s 26ms/step
+++++++++++ [12_0] | 2 ++++++++++++++++++
Elapsed time:  0.04156843300006585
Stopped at epoch:  0
1/1 [==============================] - 0s 25ms/step
+++++++++++ [12_0] | 3 ++++++++++++++++++
Elapsed time:  0.05741204799960542
Stopped at epoch:  0
1/1 [==============================] - 0s 27ms/step
+++++++++++ [12_0] | 4 ++++++++++++++++++
Elapsed time:  0.04844604400022945
Stopped at epoch:  0
1/1 [==============================] - 0s 27ms/step
+++++++++++ [12_0] | 5 ++++++++++++++++++
Elapsed time:  0.09164783999949577
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
+++++++++++ [12_0] | 6 ++++++++++++++++++
Elapsed time:  0.049824167999759084
Stopped at epoch:  0
1/1 [========================